In [1]:
# Based on Tensorflow example notebook:
# https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/3_NeuralNetworks/multilayer_perceptron.ipynb

In [2]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [3]:
# Generate tain and test data
X, Y = make_classification(n_samples=50000, n_features=10, n_informative=8, 
                           n_redundant=0, n_clusters_per_class=2)
Y = np.array([Y, -(Y-1)]).T  # The model currently needs one column for each class
X, X_test, Y, Y_test = train_test_split(X, Y)

(array([[-1.24430342, -0.73149354,  0.22683871, ..., -1.00641359,
          2.62743933, -0.24182965],
        [ 2.88785633,  0.9330167 , -0.15666524, ...,  2.72986959,
          1.35183936,  0.81295629],
        [-0.542084  ,  0.11766339, -1.18406477, ...,  0.91742041,
         -0.43773777, -0.67927821],
        ..., 
        [-0.91362092, -0.22572356,  1.51677133, ...,  3.07651302,
         -1.31009814, -2.55877362],
        [ 0.14101135, -0.01963226, -1.10042978, ...,  1.64374441,
         -2.12731293,  0.10212104],
        [ 0.35258824, -0.97394849,  0.23003364, ..., -2.05106547,
         -0.76046689,  2.25366296]]), array([[0, 1],
        [1, 0],
        [1, 0],
        ..., 
        [0, 1],
        [1, 0],
        [0, 1]]))

In [7]:
# Parameters
learning_rate = 0.001
training_epochs = 100
batch_size = 100
display_step = 1


# Network Parameters
n_hidden_1 = 10 # 1st layer number of features
n_hidden_2 = 10 # 2nd layer number of features
n_input = 10 # Number of feature
n_classes = 2 # Number of classes to predict


# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

In [9]:
# Launch the graph

with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(X)/batch_size)
        X_batches = np.array_split(X, total_batch)
        Y_batches = np.array_split(Y, total_batch)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = X_batches[i], Y_batches[i]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: X_test, y: Y_test}))
    global result 
    result = tf.argmax(pred, 1).eval({x: X_test, y: Y_test})

Epoch: 0001 cost= 3.919787795
Epoch: 0002 cost= 0.862024251
Epoch: 0003 cost= 0.575061621
Epoch: 0004 cost= 0.480831655
Epoch: 0005 cost= 0.424035816
Epoch: 0006 cost= 0.381602094
Epoch: 0007 cost= 0.345609184
Epoch: 0008 cost= 0.313407161
Epoch: 0009 cost= 0.284637511
Epoch: 0010 cost= 0.260260395
Epoch: 0011 cost= 0.239514138
Epoch: 0012 cost= 0.221683116
Epoch: 0013 cost= 0.206271455
Epoch: 0014 cost= 0.192959751
Epoch: 0015 cost= 0.181039112
Epoch: 0016 cost= 0.170597538
Epoch: 0017 cost= 0.161407666
Epoch: 0018 cost= 0.153532790
Epoch: 0019 cost= 0.146593357
Epoch: 0020 cost= 0.140314132
Epoch: 0021 cost= 0.134517673
Epoch: 0022 cost= 0.129470455
Epoch: 0023 cost= 0.124904774
Epoch: 0024 cost= 0.120846928
Epoch: 0025 cost= 0.117237567
Epoch: 0026 cost= 0.113995399
Epoch: 0027 cost= 0.110859419
Epoch: 0028 cost= 0.107819887
Epoch: 0029 cost= 0.105043510
Epoch: 0030 cost= 0.102532665
Epoch: 0031 cost= 0.100127526
Epoch: 0032 cost= 0.097815579
Epoch: 0033 cost= 0.095725833
Epoch: 003